In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

import category_encoders as ce

root='C:/Users/luthf/Documents/DS/BasketAnalysis/'

In [4]:
aisles = pd.read_csv(root + 'aisles.csv')

departments = pd.read_csv(root + 'departments.csv')

orders = pd.read_csv(root + 'orders.csv', 
                 dtype={
                        'order_id': np.int32,
                        'user_id': np.int64,
                        'eval_set': 'category',
                        'order_number': np.int16,
                        'order_dow': np.int8,
                        'order_hour_of_day': np.int8,
                        'days_since_prior_order': np.float32})

order_products_prior = pd.read_csv(root + 'order_products__prior.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

order_products_train = pd.read_csv(root + 'order_products__train.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})
products = pd.read_csv(root + 'products.csv')

In [5]:
df_feature=order_products_prior.merge(orders,on='order_id',how='inner')
df_feature=df_feature.merge(products,on='product_id',how='left')
df_feature.head(10)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13
5,2,17794,6,1,202279,prior,3,5,9,8.0,Carrots,83,4
6,2,40141,7,1,202279,prior,3,5,9,8.0,Original Unflavored Gelatine Mix,105,13
7,2,1819,8,1,202279,prior,3,5,9,8.0,All Natural No Stir Creamy Almond Butter,88,13
8,2,43668,9,0,202279,prior,3,5,9,8.0,Classic Blend Cole Slaw,123,4
9,3,33754,1,1,205970,prior,16,5,17,12.0,Total 2% with Strawberry Lowfat Greek Strained...,120,16


In [6]:
df_feature.shape

(32434489, 13)

In [7]:
df_feature['buy_product_times']=df_feature.groupby(['user_id','product_id']).cumcount()+1

In [8]:
df_feature.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [9]:
dict1_agg={'add_to_cart_order':{'mean_add_to_cart':'mean'},
       'reordered' : {'total_order':'count','total_reordered':'sum','reorderd_percentage':'mean'},
       'user_id': {'unique_user':{lambda x:x.nunique()}},
       'buy_product_times':{'first_buy': lambda x:sum(x==1),'second_buy':lambda x:sum(x==2)}}
       
      
      
    

In [10]:
#.agg(new_col_name=('col_name','agg_func')
prod_level=df_feature.groupby('product_id').agg(mean_add_to_cart=('add_to_cart_order','mean'),
                                            total_order=('reordered','count'),
                                            total_reorder=('reordered','sum'),
                                            reordered_percentage=('reordered','mean'),
                                            unique_user=('user_id',lambda x:x.nunique()),
                                            first_buy=('buy_product_times',lambda x:sum(x==1)),
                                            second_buy=('buy_product_times',lambda x:sum(x==2)))
                                               
prod_level.reset_index(inplace = True)
prod_level.head()

,product_id,mean_add_to_cart,total_order,total_reorder,reordered_percentage,unique_user,first_buy,second_buy
0,1,5.801836,1852,1136.0,0.613391,716,716,276
1,2,9.888889,90,12.0,0.133333,78,78,8
2,3,6.415162,277,203.0,0.732852,74,74,36
3,4,9.507599,329,147.0,0.446809,182,182,64
4,5,6.466667,15,9.0,0.600000,6,6,4


In [11]:
prod_level.head()

,product_id,mean_add_to_cart,total_order,total_reorder,reordered_percentage,unique_user,first_buy,second_buy
0,1,5.801836,1852,1136.0,0.613391,716,716,276
1,2,9.888889,90,12.0,0.133333,78,78,8
2,3,6.415162,277,203.0,0.732852,74,74,36
3,4,9.507599,329,147.0,0.446809,182,182,64
4,5,6.466667,15,9.0,0.600000,6,6,4


In [12]:
#KITA MASUK KE AISLE AND DEPARTMENT LEVEL

In [13]:
#.agg(new_col_name=('col_name','agg_func')
aisle_level=df_feature.groupby('aisle_id').agg(aisle_mean_add_to_cart=('add_to_cart_order','mean'),
                                               aisle_std_add_to_cart=('add_to_cart_order',np.std),
                                               aisle_total_order=('reordered','count'),
                                            aisle_total_reorder=('reordered','sum'),
                                            aisle_reordered_percentage=('reordered','mean'),
                                            aisle_unique_user=('user_id',lambda x:x.nunique()))
                                               
aisle_level.reset_index(inplace = True)
aisle_level.head()

,aisle_id,aisle_mean_add_to_cart,aisle_std_add_to_cart,aisle_total_order,aisle_total_reorder,aisle_reordered_percentage,aisle_unique_user
0,1,8.167640,7.104166,71928,42912.0,0.596597,20711
1,2,9.275497,7.473802,82491,40365.0,0.489326,31222
2,3,9.571935,7.899672,456386,272922.0,0.598007,63592
3,4,10.161450,7.745705,200687,98243.0,0.489533,53892
4,5,10.297600,8.187047,62510,17542.0,0.280627,32312


In [14]:
#.agg(new_col_name=('col_name','agg_func')
dept_level=df_feature.groupby('department_id').agg(department_mean_add_to_cart=('add_to_cart_order','mean'),
                                               department_std_add_to_cart=('add_to_cart_order',np.std),
                                            department_reordered_percentage=('reordered','mean'),
                                            department_unique_user=('user_id',lambda x:x.nunique()))
                                               
dept_level.reset_index(inplace = True)
dept_level.head()

,department_id,department_mean_add_to_cart,department_std_add_to_cart,department_reordered_percentage,department_unique_user
0,1,8.996414,7.393502,0.541885,163233
1,2,8.277645,7.526272,0.407980,17875
2,3,8.084397,6.904849,0.628141,140612
3,4,8.022875,6.658899,0.649913,193237
4,5,5.428346,5.778253,0.569924,15798


In [15]:
prod_level = prod_level.merge(products, on = 'product_id', how = 'left')
prod_level = prod_level.merge(aisle_level, on = 'aisle_id', how = 'left')
prod_level = prod_level.merge(aisles, on = 'aisle_id', how = 'left')
prod_level = prod_level.merge(dept_level, on = 'department_id', how = 'left')
prod_level = prod_level.merge(departments, on = 'department_id', how = 'left')
prod_level.head()

,product_id,mean_add_to_cart,total_order,total_reorder,reordered_percentage,unique_user,first_buy,second_buy,product_name,aisle_id,...,aisle_total_order,aisle_total_reorder,aisle_reordered_percentage,aisle_unique_user,aisle,department_mean_add_to_cart,department_std_add_to_cart,department_reordered_percentage,department_unique_user,department
0,1,5.801836,1852,1136.0,0.613391,716,716,276,Chocolate Sandwich Cookies,61,...,234065,128431.0,0.548698,54202,cookies cakes,9.187743,7.692492,0.574180,174219,snacks
1,2,9.888889,90,12.0,0.133333,78,78,8,All-Seasons Salt,104,...,212092,32321.0,0.152391,76402,spices seasonings,9.593425,7.875241,0.346721,172755,pantry
2,3,6.415162,277,203.0,0.732852,74,74,36,Robust Golden Unsweetened Oolong Tea,94,...,249341,131556.0,0.527615,53197,tea,6.976699,6.711172,0.653460,172795,beverages
3,4,9.507599,329,147.0,0.446809,182,182,64,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,...,390299,217262.0,0.556655,58749,frozen meals,8.996414,7.393502,0.541885,163233,frozen
4,5,6.466667,15,9.0,0.600000,6,6,4,Green Chile Anytime Sauce,5,...,62510,17542.0,0.280627,32312,marinades meat preparation,9.593425,7.875241,0.346721,172755,pantry


In [16]:
prod_level.drop(['product_name', 'aisle_id', 'department_id'], axis = 1, inplace = True)
prod_level.head()

,product_id,mean_add_to_cart,total_order,total_reorder,reordered_percentage,unique_user,first_buy,second_buy,aisle_mean_add_to_cart,aisle_std_add_to_cart,aisle_total_order,aisle_total_reorder,aisle_reordered_percentage,aisle_unique_user,aisle,department_mean_add_to_cart,department_std_add_to_cart,department_reordered_percentage,department_unique_user,department
0,1,5.801836,1852,1136.0,0.613391,716,716,276,9.253092,7.845107,234065,128431.0,0.548698,54202,cookies cakes,9.187743,7.692492,0.574180,174219,snacks
1,2,9.888889,90,12.0,0.133333,78,78,8,9.996181,7.994790,212092,32321.0,0.152391,76402,spices seasonings,9.593425,7.875241,0.346721,172755,pantry
2,3,6.415162,277,203.0,0.732852,74,74,36,8.519846,7.818249,249341,131556.0,0.527615,53197,tea,6.976699,6.711172,0.653460,172795,beverages
3,4,9.507599,329,147.0,0.446809,182,182,64,9.207741,7.435740,390299,217262.0,0.556655,58749,frozen meals,8.996414,7.393502,0.541885,163233,frozen
4,5,6.466667,15,9.0,0.600000,6,6,4,10.297600,8.187047,62510,17542.0,0.280627,32312,marinades meat preparation,9.593425,7.875241,0.346721,172755,pantry


In [17]:
encoder= ce.BinaryEncoder(cols=['aisle', 'department'],return_df=True)

In [18]:
prod_level = encoder.fit_transform(prod_level)
prod_level.head()

,product_id,mean_add_to_cart,total_order,total_reorder,reordered_percentage,unique_user,first_buy,second_buy,aisle_mean_add_to_cart,aisle_std_add_to_cart,...,aisle_7,department_mean_add_to_cart,department_std_add_to_cart,department_reordered_percentage,department_unique_user,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136.0,0.613391,716,716,276,9.253092,7.845107,...,1,9.187743,7.692492,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12.0,0.133333,78,78,8,9.996181,7.994790,...,0,9.593425,7.875241,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203.0,0.732852,74,74,36,8.519846,7.818249,...,1,6.976699,6.711172,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147.0,0.446809,182,182,64,9.207741,7.435740,...,0,8.996414,7.393502,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9.0,0.600000,6,6,4,10.297600,8.187047,...,1,9.593425,7.875241,0.346721,172755,0,0,0,1,0


In [19]:
prod_level.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49677 entries, 0 to 49676
Data columns (total 31 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   product_id                       49677 non-null  uint64 
 1   mean_add_to_cart                 49677 non-null  float64
 2   total_order                      49677 non-null  int64  
 3   total_reorder                    49677 non-null  float64
 4   reordered_percentage             49677 non-null  float64
 5   unique_user                      49677 non-null  int64  
 6   first_buy                        49677 non-null  int64  
 7   second_buy                       49677 non-null  int64  
 8   aisle_mean_add_to_cart           49677 non-null  float64
 9   aisle_std_add_to_cart            49677 non-null  float64
 10  aisle_total_order                49677 non-null  int64  
 11  aisle_total_reorder              49677 non-null  float64
 12  aisle_reordered_pe

In [20]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [21]:
df_feature.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [22]:
df_feature.isnull().any()

order_id                  False
product_id                False
add_to_cart_order         False
reordered                 False
user_id                   False
eval_set                  False
order_number              False
order_dow                 False
order_hour_of_day         False
days_since_prior_order     True
product_name              False
aisle_id                  False
department_id             False
buy_product_times         False
dtype: bool

In [23]:
df_feature.days_since_prior_order =df_feature.days_since_prior_order.fillna(0)

In [24]:
#.agg(new_col_name=('col_name','agg_func')
user_level=df_feature.groupby('user_id').agg(user_avg_dow_order=('order_dow','mean'),
                                              user_std_dow_order=('order_dow',np.std),
                                              user_avg_hour_order=('order_hour_of_day','mean'),
                                              user_std_hour_order=('order_hour_of_day',np.std),
                                              user_avg_prior_order=('days_since_prior_order','mean'),
                                              user_std_prior_order=('days_since_prior_order',np.std),
                                               user_total_order=('order_number',lambda x:x.nunique()),
                                              user_total_product_order=('product_id','count'),
                                              user__unique_product_order=('product_id',lambda x:x.nunique()),
                                              user_total_reorder=('reordered','sum'),
                                              user_overal_reorder=('reordered','mean'))
                                                                               
user_level.reset_index(inplace = True)
user_level.head()

,user_id,user_avg_dow_order,user_std_dow_order,user_avg_hour_order,user_std_hour_order,user_avg_prior_order,user_std_prior_order,user_total_order,user_total_product_order,user__unique_product_order,user_total_reorder,user_overal_reorder
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41.0,0.694915
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93.0,0.476923
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867395,12,88,33,55.0,0.625000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1.0,0.055556
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14.0,0.378378


In [25]:
user_level.shape

(206209, 12)

In [26]:
#.agg(new_col_name=('col_name','agg_func')
user_level2=df_feature.groupby(['user_id','order_number']).agg(avg_order_size=('reordered','count'),
                                                              mean_reordered_item=('reordered','mean'))
                                              
                                                                               
user_level2.reset_index(inplace = True)
user_level2.head()

,user_id,order_number,avg_order_size,mean_reordered_item
0,1,1,5,0.000
1,1,2,6,0.500
2,1,3,5,0.600
3,1,4,5,1.000
4,1,5,8,0.625


In [27]:
user_level2.shape

(3214874, 4)

In [28]:
user_level3 = user_level2.groupby('user_id').agg(avg_order_size=('avg_order_size','mean'),
                                                mean_reorder_size=('mean_reordered_item','mean'))

user_level3 = user_level3.reset_index()
user_level3.head()

,user_id,avg_order_size,mean_reorder_size
0,1,5.900000,0.705833
1,2,13.928571,0.447961
2,3,7.333333,0.658817
3,4,3.600000,0.028571
4,5,9.250000,0.377778


In [29]:
user_level3.shape

(206209, 3)

In [30]:
user_level=user_level.merge(user_level3,on='user_id',how='left')
user_level.head()

,user_id,user_avg_dow_order,user_std_dow_order,user_avg_hour_order,user_std_hour_order,user_avg_prior_order,user_std_prior_order,user_total_order,user_total_product_order,user__unique_product_order,user_total_reorder,user_overal_reorder,avg_order_size,mean_reorder_size
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41.0,0.694915,5.900000,0.705833
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93.0,0.476923,13.928571,0.447961
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867395,12,88,33,55.0,0.625000,7.333333,0.658817
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1.0,0.055556,3.600000,0.028571
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14.0,0.378378,9.250000,0.377778


In [31]:
last_three_orders = user_level2.groupby('user_id')['order_number'].nlargest(3).reset_index()
last_three_orders

,user_id,level_1,order_number
0,1,9,10
1,1,8,9
2,1,7,8
3,2,23,14
4,2,22,13
...,...,...,...
618622,206208,3214859,48
618623,206208,3214858,47
618624,206209,3214873,13
618625,206209,3214872,12


In [32]:
last_three_orders = user_level2.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_three_orders.head()

,user_id,order_number,avg_order_size,mean_reordered_item,level_1
0,1,8,6,0.666667,7
1,1,9,6,1.000000,8
2,1,10,9,0.666667,9
3,2,12,19,0.578947,21
4,2,13,9,0.000000,22


In [33]:
last_three_orders['rank'] = last_three_orders.groupby("user_id")["order_number"].rank("dense", ascending=True)
last_three_orders.head(20)

,user_id,order_number,avg_order_size,mean_reordered_item,level_1,rank
0,1,8,6,0.666667,7,1.0
1,1,9,6,1.000000,8,2.0
2,1,10,9,0.666667,9,3.0
3,2,12,19,0.578947,21,1.0
4,2,13,9,0.000000,22,2.0
5,2,14,16,0.625000,23,3.0
6,3,10,6,0.833333,33,1.0
7,3,11,5,1.000000,34,2.0
8,3,12,6,1.000000,35,3.0
9,4,3,7,0.142857,38,1.0


In [34]:
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], \
                                                 values=['avg_order_size', 'mean_reordered_item']).\
                                                reset_index(drop = False)
last_order_feats.columns = ['user_id','orders_3', 'orders_2', 'orders_1', 'reorder_3', 'reorder_2', 'reorder_1']
last_order_feats.head()

,user_id,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,6,6,9,0.666667,1.0,0.666667
1,2,19,9,16,0.578947,0.0,0.625000
2,3,6,5,6,0.833333,1.0,1.000000
3,4,7,2,3,0.142857,0.0,0.000000
4,5,9,5,12,0.444444,0.4,0.666667


In [35]:
user_level=user_level.merge(last_order_feats,on='user_id',how='left')
user_level.head()

,user_id,user_avg_dow_order,user_std_dow_order,user_avg_hour_order,user_std_hour_order,user_avg_prior_order,user_std_prior_order,user_total_order,user_total_product_order,user__unique_product_order,user_total_reorder,user_overal_reorder,avg_order_size,mean_reorder_size,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41.0,0.694915,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93.0,0.476923,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867395,12,88,33,55.0,0.625000,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1.0,0.055556,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14.0,0.378378,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


In [36]:
user_level.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206209 entries, 0 to 206208
Data columns (total 20 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   user_id                     206209 non-null  int64  
 1   user_avg_dow_order          206209 non-null  float64
 2   user_std_dow_order          206209 non-null  float64
 3   user_avg_hour_order         206209 non-null  float64
 4   user_std_hour_order         206209 non-null  float64
 5   user_avg_prior_order        206209 non-null  float32
 6   user_std_prior_order        206209 non-null  float32
 7   user_total_order            206209 non-null  int16  
 8   user_total_product_order    206209 non-null  int64  
 9   user__unique_product_order  206209 non-null  uint16 
 10  user_total_reorder          206209 non-null  float64
 11  user_overal_reorder         206209 non-null  float64
 12  avg_order_size              206209 non-null  float64
 13  mean_reorder_s

### User-Product Level

In [37]:
#.agg(new_col_name=('col_name','agg_func')
user_product_level=df_feature.groupby(['user_id','product_id']).agg(avg_add_to_cart_by_user=('add_to_cart_order','mean'),
                                                                    avg_day_since_prior_order=('days_since_prior_order','mean'),
                                                                    user_total_order_product=('reordered','count'),
                                                                    user_total_reorder_product=('reordered','sum'),
                                                                    user_percentage_reorder_product=('reordered','mean'),
                                                                    last_order_in=('order_number','max')
                                                                   )
                                                                               
user_product_level.reset_index(inplace = True)
user_product_level.head()

,user_id,product_id,avg_add_to_cart_by_user,avg_day_since_prior_order,user_total_order_product,user_total_reorder_product,user_percentage_reorder_product,last_order_in
0,1,196,1.400000,17.600000,10,9,0.900000,10
1,1,10258,3.333333,19.555555,9,8,0.888889,10
2,1,10326,5.000000,28.000000,1,0,0.000000,5
3,1,12427,3.300000,17.600000,10,9,0.900000,10
4,1,13032,6.333333,21.666666,3,2,0.666667,10


In [38]:
last_order=df_feature.merge(last_three_orders,on=['user_id','order_number'],how='inner')

In [39]:
last_order.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,buy_product_times,avg_order_size,mean_reordered_item,level_1,rank
0,7,34050,1,0,142903,prior,11,2,14,30.0,Orange Juice,31,7,1,2,0.000000,2231251,2.0
1,7,46802,2,0,142903,prior,11,2,14,30.0,Pineapple Chunks,116,1,1,2,0.000000,2231251,2.0
2,14,20392,1,1,18194,prior,49,3,15,3.0,Hair Bender Whole Bean Coffee,26,7,1,11,0.818182,282882,1.0
3,14,27845,2,1,18194,prior,49,3,15,3.0,Organic Whole Milk,84,16,1,11,0.818182,282882,1.0
4,14,162,3,1,18194,prior,49,3,15,3.0,Organic Mini Homestyle Waffles,52,1,1,11,0.818182,282882,1.0


In [40]:
last_order['rank'] = last_order.groupby(['user_id', 'product_id'])['order_number'].rank("dense", ascending=True)

In [41]:
product_purchase_history = last_order.pivot_table(index = ['user_id', 'product_id'],\
                                                   columns='rank', values = 'reordered').reset_index()
product_purchase_history.columns = ['user_id', 'product_id', 'is_reorder_3', 'is_reorder_2', 'is_reorder_1']
product_purchase_history.fillna(0, inplace = True)
product_purchase_history.head()

,user_id,product_id,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.0,1.0,1.0
1,1,10258,1.0,1.0,1.0
2,1,12427,1.0,1.0,1.0
3,1,13032,1.0,0.0,0.0
4,1,25133,1.0,1.0,1.0


In [42]:
user_product_feats = user_product_level.merge(product_purchase_history, on=['user_id', 'product_id'], how = 'left')
user_product_feats.head()

,user_id,product_id,avg_add_to_cart_by_user,avg_day_since_prior_order,user_total_order_product,user_total_reorder_product,user_percentage_reorder_product,last_order_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.400000,17.600000,10,9,0.900000,10,1.0,1.0,1.0
1,1,10258,3.333333,19.555555,9,8,0.888889,10,1.0,1.0,1.0
2,1,10326,5.000000,28.000000,1,0,0.000000,5,NaN,NaN,NaN
3,1,12427,3.300000,17.600000,10,9,0.900000,10,1.0,1.0,1.0
4,1,13032,6.333333,21.666666,3,2,0.666667,10,1.0,0.0,0.0


In [43]:
user_product_feats.isnull().sum()

user_id                                  0
product_id                               0
avg_add_to_cart_by_user                  0
avg_day_since_prior_order                0
user_total_order_product                 0
user_total_reorder_product               0
user_percentage_reorder_product          0
last_order_in                            0
is_reorder_3                       8382738
is_reorder_2                       8382738
is_reorder_1                       8382738
dtype: int64

In [44]:
user_product_feats.fillna(0, inplace = True)

### saving features

In [54]:
prod_level.to_pickle(root + 'prod_level.pkl')
user_level.to_pickle(root +'user_level.pkl')
user_product_feats.to_pickle(root +'user_product_features.pkl')

In [55]:
df = pd.read_pickle(root+'prod_level.pkl')
df.head()

,product_id,mean_add_to_cart,total_order,total_reorder,reordered_percentage,unique_user,first_buy,second_buy,aisle_mean_add_to_cart,aisle_std_add_to_cart,...,aisle_7,department_mean_add_to_cart,department_std_add_to_cart,department_reordered_percentage,department_unique_user,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136.0,0.613391,716,716,276,9.253092,7.845107,...,1,9.187743,7.692492,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12.0,0.133333,78,78,8,9.996181,7.994790,...,0,9.593425,7.875241,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203.0,0.732852,74,74,36,8.519846,7.818249,...,1,6.976699,6.711172,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147.0,0.446809,182,182,64,9.207741,7.435740,...,0,8.996414,7.393502,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9.0,0.600000,6,6,4,10.297600,8.187047,...,1,9.593425,7.875241,0.346721,172755,0,0,0,1,0


In [56]:
df = pd.read_pickle(root+'prod_level.pkl')
df.head()

,product_id,mean_add_to_cart,total_order,total_reorder,reordered_percentage,unique_user,first_buy,second_buy,aisle_mean_add_to_cart,aisle_std_add_to_cart,...,aisle_7,department_mean_add_to_cart,department_std_add_to_cart,department_reordered_percentage,department_unique_user,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136.0,0.613391,716,716,276,9.253092,7.845107,...,1,9.187743,7.692492,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12.0,0.133333,78,78,8,9.996181,7.994790,...,0,9.593425,7.875241,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203.0,0.732852,74,74,36,8.519846,7.818249,...,1,6.976699,6.711172,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147.0,0.446809,182,182,64,9.207741,7.435740,...,0,8.996414,7.393502,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9.0,0.600000,6,6,4,10.297600,8.187047,...,1,9.593425,7.875241,0.346721,172755,0,0,0,1,0


In [57]:
df = pd.read_pickle(root+'user_product_features.pkl')
df.head()

,user_id,product_id,avg_add_to_cart_by_user,avg_day_since_prior_order,user_total_order_product,user_total_reorder_product,user_percentage_reorder_product,last_order_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.400000,17.600000,10,9,0.900000,10,1.0,1.0,1.0
1,1,10258,3.333333,19.555555,9,8,0.888889,10,1.0,1.0,1.0
2,1,10326,5.000000,28.000000,1,0,0.000000,5,0.0,0.0,0.0
3,1,12427,3.300000,17.600000,10,9,0.900000,10,1.0,1.0,1.0
4,1,13032,6.333333,21.666666,3,2,0.666667,10,1.0,0.0,0.0


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 11 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   user_id                          int64  
 1   product_id                       uint64 
 2   avg_add_to_cart_by_user          float64
 3   avg_day_since_prior_order        float32
 4   user_total_order_product         int64  
 5   user_total_reorder_product       int8   
 6   user_percentage_reorder_product  float64
 7   last_order_in                    int16  
 8   is_reorder_3                     float64
 9   is_reorder_2                     float64
 10  is_reorder_1                     float64
dtypes: float32(1), float64(5), int16(1), int64(2), int8(1), uint64(1)
memory usage: 1002.6 MB
